In [5]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.io.input import loadMidiFile
from src.io.record_midi import MidiInput
from src.io.output import pianoRoll, playPrettyMidi
from src.io.conversion import note_seq_to_pretty_midi
from src import analysis
from src.adaptation import Adaptation
from src.datatypes.melody_data import MelodyData
from definitions import SequenceType
from src.generation.musicvae_generator import MusicVAEGenerator, Checkpoint

# Initialize the generation model
(takes up to 3 minutes)

In [16]:
vae = MusicVAEGenerator()

[GEN] Initializing Music VAE with checkpoint 'cat-mel_2bar_big'...
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

INFO:tensorflow:Unbundling checkpoint.
INFO:tensorflow:Restoring parameters from C:\Users\Eric\AppData\Local\Temp\tmpzkobgxcy\cat-mel_2bar_big.ckpt
[GEN] 

# Preparation of the Adaptation Pipeline
## Instantiate Adaptation Object


In [31]:
a = Adaptation()

## Print available adaptation operations

In [32]:
for op in a.available_operations:
    print(op.__name__, ":", op.__doc__)

StartAndEndOnCTOperation : Creates a chord based on the estimated key of the control sequence and transposes the first and last note of the base sequence so they are chord tones (e.g. 'c', 'e' or 'g' for a C major chord).
TransposeNotesOperation : Estimates the key of both sequences and transposes every single note of the base sequence separately to the closest pitch that fits the key of the control sequence.
TransposeSequenceOperation : Estimates the key of both sequences and transposes the base sequence to be in the same key as the control sequence.


## Construct a pipeline

In [33]:
a.construct_pipeline(['StartAndEndOnCTOperation', 'TransposeNotesOperation'])

# Record input melody with a length of 4 bars
+ using metronome but no playback

In [40]:
recorder = MidiInput(tempo=120, playback=False)
recorded_input = recorder.recordBars(4)

[IO] "MPK Mini Mk II 0" connected as input port
[IO] WARNING: Input port "MPK Mini Mk II 0" could not be closed. Most likely, it is already closed.
[IO] NOW RECORDING ...

[IO] recording successful.


# Generate base sequence for adaptation
+ parameters: number of beats and temperature

In [42]:
gen_base = vae.generate(16, 0.2)

# Create MelodyData objects and start adaptation

In [44]:
mel_input = MelodyData(recorded_input, SequenceType.REC_INPUT, {})
mel_gen_base = MelodyData(note_seq_to_pretty_midi(gen_base['sequence']), SequenceType.GEN_BASE, { 'generation': gen_base['meta'] })

result = a.adapt(mel_gen_base, mel_input)

# Print & play the results

In [45]:
pianoRoll(mel_gen_base.sequence)
pianoRoll(result.sequence)
print(result.meta)

Loading BokehJS ...

Loading BokehJS ...

{'generation': {'gen_dur': 20.904886722564697, 'model': 'MusicVAE', 'checkpoint': 'cat-mel_2bar_big', 'temperature': 0.2}, 'adaptation_steps': [{'name': 'StartAndEndOnCTOperation', 'duration': 0.16910433769226074, 'intermediate_result': <music21.stream.Score 0x19373993308>}, {'name': 'TransposeNotesOperation', 'duration': 0.12018799781799316, 'intermediate_result': <music21.stream.Score 0x19373993308>}], 'adaptation_duration': 0.2892923355102539}


In [46]:
playPrettyMidi(mel_gen_base.sequence)

In [47]:
playPrettyMidi(result.sequence)